Installing Necessary libraries

In [1]:
!pip install -q openai langchain playwright beautifulsoup4 tiktoken
!playwright install
!playwright install-deps

You should consider upgrading via the '/Users/edwarddennispraveen/Documents/azure_meetup/meetup/bin/python3 -m pip install --upgrade pip' command.


Setting up OpenAI

In [2]:
import os
from langchain.chat_models import ChatOpenAI
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

/Users/edwarddennispraveen/Documents/azure_meetup/meetup/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Importing for async issues

In [3]:
import nest_asyncio
nest_asyncio.apply()

Asynchronous Loading of Web Pages

In [4]:
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import BeautifulSoupTransformer
loader = AsyncChromiumLoader(["https://www.flipkart.com/search?q=iphone"])
docs = loader.load()

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
        docs, tags_to_extract=["div","span","h2","a"]
    )

In [5]:
docs_transformed

[Document(page_content='Explore (/plus) Plus Login (/account/login?ret=/search%3Fq%3Diphone) LOGIN New customer? Sign Up Become a Seller More Cart Electronics TVs & Appliances Men Women Baby & Kids Home & Furniture Sports, Books & More Flights (/travel/flights?otracker=nmenu_Flights) Offer Zone (/offers-list/top-deals?screen=dynamic&pk=themeViews%3DDT-OMU-A2%3ADT-OMU~widgetType%3DdealCard~contentType%3Dneo&otracker=nmenu_offer-zone) Filters CATEGORIES Mobiles & Accessories (/mobiles-accessories/pr?sid=tyy&q=iphone&otracker=categorytree) Mobiles (/mobiles/pr?sid=tyy,4io&q=iphone&otracker=categorytree) Price . . . . . . Min ₹10000 ₹15000 ₹20000 ₹30000 to ₹10000 ₹15000 ₹20000 ₹30000 ₹30000+ Brand APPLE ? Customer Ratings 4★ & above 3★ & above GST Invoice Available RAM 1GB and Below Internal Storage Battery Capacity Screen Size Primary Camera Secondary Camera Processor Brand Speciality Resolution Type Operating System Network Type Sim Type Offers Special Price Buy More, Save More No Cost E

Splitting Text into Manageable Chunks

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=0)
splits = splitter.split_documents(docs_transformed)

In [8]:
splits

[Document(page_content='Explore (/plus) Plus Login (/account/login?ret=/search%3Fq%3Diphone) LOGIN New customer? Sign Up Become a Seller More Cart Electronics TVs & Appliances Men Women Baby & Kids Home & Furniture Sports, Books & More Flights (/travel/flights?otracker=nmenu_Flights) Offer Zone (/offers-list/top-deals?screen=dynamic&pk=themeViews%3DDT-OMU-A2%3ADT-OMU~widgetType%3DdealCard~contentType%3Dneo&otracker=nmenu_offer-zone) Filters CATEGORIES Mobiles & Accessories (/mobiles-accessories/pr?sid=tyy&q=iphone&otracker=categorytree) Mobiles (/mobiles/pr?sid=tyy,4io&q=iphone&otracker=categorytree) Price . . . . . . Min ₹10000 ₹15000 ₹20000 ₹30000 to ₹10000 ₹15000 ₹20000 ₹30000 ₹30000+ Brand APPLE ? Customer Ratings 4★ & above 3★ & above GST Invoice Available RAM 1GB and Below Internal Storage Battery Capacity Screen Size Primary Camera Secondary Camera Processor Brand Speciality Resolution Type Operating System Network Type Sim Type Offers Special Price Buy More, Save More No Cost E

Defining Data Extraction Schema

In [9]:
from langchain.chains import create_extraction_chain
schema = {
    "properties": {
        "product_title": {"type": "string"},
        "product_mrp": {"type": "integer"},
        "product_description":{"type": "array", "items": {"type": "string"}},
        "product_reviews_count":{"type": "string"}
    },
    "required": ["product_title","product_mrp","product_description"],
}
def extract(content: str, schema: dict):
    return create_extraction_chain(schema=schema, llm=llm).run(content)

Data Extraction

In [10]:
extracted_content = extract(schema=schema, content=splits[0].page_content)

/Users/edwarddennispraveen/Documents/azure_meetup/meetup/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
extracted_content

[{'product_title': 'APPLE iPhone 15 (Black, 128 GB)',
  'product_mrp': 66999,
  'product_description': ['128 GB ROM',
   '15.49 cm (6.1 inch) Super Retina XDR Display',
   '48MP + 12MP | 12MP Front Camera',
   'A16 Bionic Chip, 6 Core Processor',
   '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories'],
  'product_reviews_count': '3,976 Ratings & 259 Reviews'},
 {'product_title': 'APPLE iPhone 15 (Blue, 128 GB)',
  'product_mrp': 66999,
  'product_description': ['128 GB ROM',
   '15.49 cm (6.1 inch) Super Retina XDR Display',
   '48MP + 12MP | 12MP Front Camera',
   'A16 Bionic Chip, 6 Core Processor',
   '1 Year Warranty for Phone and 6 Months Warranty for In-Box Accessories'],
  'product_reviews_count': '3,976 Ratings & 259 Reviews'},
 {'product_title': 'APPLE iPhone 14 (Midnight, 128 GB)',
  'product_mrp': 58999,
  'product_description': ['128 GB ROM',
   '15.49 cm (6.1 inch) Super Retina XDR Display',
   '12MP + 12MP | 12MP Front Camera',
   'A15 Bionic Chip, 6 